In [ ]:
import numpy as np
import sqlite3
import glob

## SET THESE VALUES!
- **DB_DIR**
    - The location of where your databases live
        - Should be either OUTPUT_DEFAULT_DIR specified at monitoring program build time, or the --ouptut-dir specfied at runtime
- **PREPROCESS_SAVE_DIR**
    - Where you want to save the compact, preprocessed numpy representations of the data from DB

In [ ]:
## SET THESE 
DB_DIR = ""
PREPROCESS_SAVE_DIR = ""

## Query Metrics

### GPU Metrics
- Ignoring the PCIe/NVLink For now

In [ ]:
GPU_MEM_UTIL_ID = 254
SMI_UTIL_ID = 203
SM_ACTIVE_ID = 1002
SM_OCCUPANCY_ID = 1003
TENSOR_ACTIVE_ID = 1004
DRAM_ACTIVE_ID = 1005

### Fetching data to analyze

In [ ]:
## Assumes that cur is already a cursor that is connected to a db

## where fields is a dict from fields: (name, dtype)
def preprocess_gpu_data(cur, fields):

    ## note: gpu data will have non-negative device_ids
    base_dt = [("timestamp", "datetime64[ns]"), ("device_id", np.uint8)]
    field_arrs = []

    print("Fetching All Fields...")
    ## cleaner to query each field individually
    ##   - instead could get better perf/potential memory savings (depending on number of fields) to 
    ##     just fetch the whole db, convert to numpy and then filter
    for k, v in fields.items():
        print(f"\t{v[0]}...")
        q = f"SELECT timestamp, device_id, value FROM Data WHERE field_id = {k}"""
        db_result = cur.execute(q).fetchall()
        q_dt = base_dt + [("value", v[1])]
        numpy_arr = np.array(db_result, dtype=q_dt)
        ## converted to numpy array so can free the list of tuples
        del db_result
        field_arrs.append((k, numpy_arr))

    ## all of these field arrs should have same timestamps and device_ids
    base_field_id = field_arrs[0][0]
    base_field_name = fields[base_field_id][0]
    num_rows = len(field_arrs[0][1])
    timestamps = field_arrs[0][1]["timestamp"]
    device_ids = field_arrs[0][1]["device_id"]

    ## Error check
    for field_num in range(1, len(field_arrs)):
        field_id = field_arrs[field_num][0]
        field_name = fields[field_id][0]
        field_arr = field_arrs[field_num][1]
        if len(field_arr) != num_rows:
            print(f"Error processing field {field_name}. Had different length [{len(field_arr)}] than {base_field_name} [{num_rows}].")
            return None
        if not np.array_equal(field_arr["timestamp"], timestamps):
            print(f"Error processing field {field_name}. Had different timestamps vs. {base_field_name}.")
            return None
        if not np.array_equal(field_arr["device_id"], device_ids):
            print(f"Error processing field {field_name}. Had different device_ids vs. {base_field_name}.")
            return None

    
    ## The field arrays are referencing the same window of time and same devices, so we can proceed
    
    ## Create output array
    print("Populating output numpy array...")
    
    base_result_dt = [("timestamp", "datetime64[ns]"), ("device_id", np.uint8)]
    result_dt = base_result_dt
    for i in range(len(field_arrs)):
        field_id = field_arrs[i][0]
        result_dt.append(fields[field_id])

    ## add diff utils if both 203 and 1002 are within the list
    if ((SMI_UTIL_ID in fields) and (SM_ACTIVE_ID in fields)):
        result_dt.append(("diff_util_metrics", np.uint8))
        
    result = np.zeros(num_rows, dtype=result_dt)

    result["timestamp"] = timestamps
    result["device_id"] = device_ids

    ## setting the values for field ids
    smi_util_ind, sm_active_ind = None, None
    for i in range(len(field_arrs)):
        field_id = field_arrs[i][0]
        field_name = fields[field_id][0]
        result[field_name] = field_arrs[i][1]["value"]
        if field_id == SMI_UTIL_ID:
            smi_util_ind = i
        if field_id == SM_ACTIVE_ID:
            sm_active_ind = i

    ## if both compute utilization metrics are being queried also include their difference (for convenience)
    if ((SMI_UTIL_ID in fields) and (SM_ACTIVE_ID in fields)):
        ## if they are both in fields than their indexes would have been set above
        result["diff_util_metrics"] = field_arrs[smi_util_ind][1]["value"] - field_arrs[sm_active_ind][1]["value"]

    ## can delete all the field_arrs now that they have been copied to the result
    for i in range(len(field_arrs)):
        orig_np_arr = field_arrs[i][1]
        del orig_np_arr
    
    return result

### Specifying what fields to retrieve to a compact numpy array for fast processing

In [ ]:
fields = {GPU_MEM_UTIL_ID: ("gpu_mem_used", np.uint8), SMI_UTIL_ID: ("smi_util", np.uint8), SM_ACTIVE_ID: ("sm_active", np.uint8), SM_OCCUPANCY_ID: ("sm_occupancy", np.uint8),
          TENSOR_ACTIVE_ID: ("tensor_active", np.uint8), DRAM_ACTIVE_ID: ("dram_active", np.uint8)}

### Iterating through all nodes and saving results to numpy files that can be easily loaded and analyzed

In [ ]:
node_db_files = sorted(glob.glob(DB_DIR + "*.db"))

In [ ]:
i = 0
for node_db_file in node_db_files:

    node_name = node_db_file.split("/")[-1].split(".")[0]
    print(f"{i}: {node_name}")
    
    ### Reading the DB into Memory 
    source_con = sqlite3.connect(node_db_file)
    con = sqlite3.connect(':memory:')
    source_con.backup(con)
    ## we copied the db to be in-memory, so we are done with the file
    source_con.close()
    cur = con.cursor()

    ## Getting the compact numpy representation
    preprocessed_result = preprocess_gpu_data(cur, fields)

    ## save results for future processing
    save_np_filename = PREPROCESS_SAVE_DIR + node_name + ".npy"
    with open(save_np_filename, 'wb') as f:
        np.save(f, preprocessed_result)

    ## can free the numpy memory before next iteration now
    del preprocessed_result

    ## disconnect from in-memory db
    con.close()
    
    i += 1